# General Setup

Install the dependencies if not installed yet

In [ ]:
using Pkg
# Pkg.add("CSVFiles")
# Pkg.add("VegaLite")
# Pkg.add("DataFrames")

using Statistics, Printf
using VegaLite, CSVFiles, DataFrames

# Compatibility
Read in `.csv` data files from what works and what does not

In [ ]:
df_rgular_exec_once = DataFrame(load("./working-dir/executes-once-analysis-regular.csv"))
df_wasabi_exec_once = DataFrame(load("./working-dir/executes-once-analysis-wasabi.csv"))
df_wastrm_exec_once = DataFrame(load("./working-dir/executes-once-analysis-wastrumentation.csv"))
"Data files read"

In [ ]:
df_rgular_executes = df_rgular_exec_once[!, :reason]
df_wasabi_executes = df_wasabi_exec_once[!, :reason]
df_wastrm_executes = df_wastrm_exec_once[!, :reason]

function known_exception(report)
    false || report == "success" || report == "timeout"
end

@assert all(known_exception, df_rgular_executes)
@assert all(report -> known_exception(report) || report == "error - Local count too large", df_wasabi_executes)
@assert all(known_exception, df_wastrm_executes)

total_input_programs = nrow(df_rgular_exec_once) # Total number of input programs

regular_success = sum(df_rgular_executes .== "success")
regular_timeout = sum(df_rgular_executes .== "timeout")
regular_timeout_report = if regular_timeout == 0 begin "" end else " ($regular_timeout timed out)" end

wasabi_success = sum(df_wasabi_executes .== "success")
wasabi_timeout = sum(df_wasabi_executes .== "timeout")
wasabi_error_r = sum(df_wasabi_executes .== "error - Local count too large")
wasabi_unsuccesful_report = if wasabi_timeout == 0 && wasabi_error_r == 0 begin
    ""
end elseif wasabi_timeout == 0 && wasabi_error_r > 0 begin
    " ($wasabi_error_r errored)"
end elseif wasabi_timeout > 0 && wasabi_error_r == 0 begin
    " ($wasabi_timeout timed out)"
end else
    " ($wasabi_timeout timed out, $wasabi_error_r errored)"
end

wastrm_success = sum(df_wastrm_executes .== "success")
wastrm_timeout = sum(df_wastrm_executes .== "timeout")
wastrm_timeout_report = if wastrm_timeout == 0 begin "" end else " ($wastrm_timeout timed out)" end

conclusion = "For the forward analysis, a total of $total_input_programs our benchmark harness succesfully executed $regular_success programs uninstrumented$regular_timeout_report, $wastrm_success after instrumentation by Wastrumentation$wastrm_timeout_report and $wasabi_success after instrumentation by Wasabi$wasabi_unsuccesful_report."

Perform a selection of all programs that passed all executions:

In [ ]:
# split 'setup' into 'platform' and 'analysis'
function split_setup(df)
    splitted = transform(
      df,
      :setup => ByRow(setup ->
        if setup === missing
          [missing, missing]
        else
          m = match(r"\[([\w-]+) - ([\w-]+)\]", setup)
          platform, analyss = [m.captures[1], m.captures[2]]
          [platform, analyss]
        end
      ) => [:platform, :analysis],
    )
    select(splitted, Not([:setup]))
end


# Code Size Study

In [ ]:
df_rgular_code_size = DataFrame(load("./working-dir/code-size-analysis-regular.csv"))
df_wasabi_code_size = DataFrame(load("./working-dir/code-size-analysis-wasabi.csv")) |> split_setup
df_wastrm_code_size = DataFrame(load("./working-dir/code-size-analysis-wastrumentation.csv")) |> split_setup
"Data files read"

## Plot Code Size Increase

In [ ]:
baseline = rename(select(df_rgular_code_size, Not([:setup])), :size_bytes => :size_bytes_baseline)

code_size_inc_transformation = [:size_bytes, :size_bytes_baseline] => ((size_bytes, size_bytes_baseline) -> size_bytes ./ size_bytes_baseline) => :code_increase

df_wasabi_code_incr = transform(innerjoin(baseline, df_wasabi_code_size, on=:input_program), code_size_inc_transformation)
df_wastrm_code_incr = transform(innerjoin(baseline, df_wastrm_code_size, on=:input_program), code_size_inc_transformation)

"Relative code size increase computed"

In [ ]:
absolute_code_size_plot = baseline |> @vlplot(
    encoding={
        x={
            field="input_program",
            type="nominal",
            axis={labelAngle=45},
            title="Input Program",
        },
        y={
            field="size_bytes_baseline",
            type="quantitative",
            axis={
                title="Program Size (bytes)",
                grid=false,
                titleFontSize=12,
                labelFontSize=10,
                labelColor="#666",  # Softer axis label color
                domainColor="#999",  # Softer axis line color
            },
            scale={
                type="log",
            },
        },
        color={
            value="#4C72B0",  # Soft blue color for bars
        },
    },
    layer=[
        {
            mark={
                type="bar",
                cornerRadiusEnd=3,  # Adds rounded corners to the top of bars
            },
        },
        {
            mark={
                type="text",
                align="center",
                baseline="middle",
                fontSize=10,
                dx=-27,
            },
            encoding={
                text={
                    field="size_bytes_baseline",
                    type="quantitative",
                    format=",.0f"
                },
                angle={"value"=90}
            }
        }
    ],
    config={
        view={stroke=:transparent},
    },
    title="Absolute Code Size per Input Program",  # Adds a descriptive title
    height=100,
)
absolute_code_size_plot |> save("./working-dir/absolute-code-size-plot.pdf")
absolute_code_size_plot

In [ ]:
df_wasabi_code_incr_forward = filter(:analysis => ==("forward"), df_wasabi_code_incr)
df_wastrm_code_incr_forward = filter(:analysis => ==("forward"), df_wastrm_code_incr)
binary_size_plot = vcat(df_wasabi_code_incr_forward, df_wastrm_code_incr_forward) |>
@vlplot(
  width=500,
  layer=[
    {
      mark="bar",
      encoding={
        color={
          field="platform",
          type="nominal",
          legend={
            title="Instrumentation Platform",
            orient="top",
          }
        },
        xOffset={
          field="platform",
          type="nominal",
        },
        x={
          field="input_program",
          type="nominal",
          axis={labelAngle="45"},
          title="Input Program",
        },
        y={
          field="code_increase",
          type="quantitative",
          axis={
            title="Program Size Increase (X)",
            grid=false,
          },
        },
      },
    },
    {
      mark="rule",
      encoding={
        y={
          datum=1,
        },
        color={value="red"}, # Color for the line
        size={value=1} # Thickness of the line
      },
    },
  ],
  config={
    view={stroke=:transparent},
  },
)

# binary_size_plot |> save("./working-dir/wasabi-wastrm-binary-size.pdf")
binary_size_plot

In [ ]:
df_rgular_code_size_baseline = transform(df_rgular_code_size, :size_bytes => :size_bytes_baseline)
df_rgular_code_size_baseline = select(df_rgular_code_size_baseline, Not([:size_bytes, :setup]))

df_wasabi_code_incr
df_wastrm_code_incr

all_code_sizes = vcat(df_wasabi_code_incr, df_wastrm_code_incr)
all_code_sizes = filter(row -> row.platform .!== missing, all_code_sizes)

In [ ]:
all_code_sizes_plot = all_code_sizes |> @vlplot(
  "transform"=[
    {
      "calculate"="substring(datum.code_increase, 0, 5)",
      "as"="code_increase_truncated"
    },
  ],
  facet={
    row={
      field="platform",
      type="nominal",
    },
  },
  spec={
    layer=[
      {
        mark="rect",encoding={
          y={
            field="analysis",
            type="nominal",
            axis={labelAngle="-30"},
          },
          x={
            field="input_program",
            type="nominal",
            axis={title="Input Program",},
            axis={labelAngle="-30"},
          },
          color={
          field="code_increase",
          type="quantitative",
          scale={
            type="log",
            scheme="blues",
          },
          legend={
            title="Code Size Increase (X)",
            orient="top",
            titleLimit=1000,
            gradientLength=540,
            titleAnchor="center",
            titleAlign="center",
          },
        },
        },
        
      },
      {
        mark={
          type="text",
          fontSize="6",
        },
        encoding={
          y={
            field="analysis",
            type="nominal",
          },
          x={
            field="input_program",
            type="nominal",
            axis={title="Input Program",},
          },
          text={
            field="code_increase_truncated",
            type="quantitative",
          },
        },
      },
    ],

  },
  config={
    axis={
      grid=true,
      tickBand="extent",
    },
  },
)

all_code_sizes_plot |> save("./working-dir/wasabi-wastrm-binary-size.pdf")
all_code_sizes_plot

In [ ]:
df_wasabi_wastrm_size_overhead = transform(
  outerjoin(
    filter(row -> row.analysis !== missing, select(rename(df_wasabi_code_size, :size_bytes => :wasabi_size_bytes), Not([:platform]))),
    select(rename(df_wastrm_code_size, :size_bytes => :wastrm_size_bytes), Not([:platform])),
    on=[:input_program, :analysis],
  ),
  [:wasabi_size_bytes, :wastrm_size_bytes]
    => ((wasabi_size_bytes, wastrm_size_bytes) -> wasabi_size_bytes ./ wastrm_size_bytes)
    => :code_size_for_wasabi_over_wastrm,
)
"Relative code size overhead computed"

In [ ]:
code_size_for_wasabi_per_time_for_wastrm_non_missing = filter(row -> row.code_size_for_wasabi_over_wastrm !== missing, df_wasabi_wastrm_size_overhead)
total_outout_cpmr_progms = nrow(code_size_for_wasabi_per_time_for_wastrm_non_missing)
wastrm_output_is_smaller = filter(row -> row.code_size_for_wasabi_over_wastrm > 1, code_size_for_wasabi_per_time_for_wastrm_non_missing)
wasabi_output_is_smaller = filter(row -> row.code_size_for_wasabi_over_wastrm < 1, code_size_for_wasabi_per_time_for_wastrm_non_missing)
wastrm_wasabi_equal_size = filter(row -> row.code_size_for_wasabi_over_wastrm === 1, code_size_for_wasabi_per_time_for_wastrm_non_missing)


sentence = "For a total of $total_outout_cpmr_progms output programs, $(nrow(wastrm_output_is_smaller)) have a smaller output size for Wastrumentation and $(nrow(wasabi_output_is_smaller)) have a smaller output size for Wasabi."
sentence
# mean(code_size_for_wasabi_per_time_for_wastrm_non_missing.code_size_for_wasabi_over_wastrm)
# std(code_size_for_wasabi_per_time_for_wastrm_non_missing.code_size_for_wasabi_over_wastrm, corrected=false)

In [ ]:
runtime_overhead_comparison_plot = df_wasabi_wastrm_size_overhead |>
@vlplot(
  "transform"=[
    {
      "calculate"="substring(datum.code_size_for_wasabi_over_wastrm, 0, 5)",
      "as"="relative_overhead_truncated"
    },
  ],
  encoding={
    y={
      field="analysis",
      type="nominal",
      axis={labelAngle="-30"},
    },
    x={
      field="input_program",
      type="nominal",
      axis={title="Input Program",labelAngle="-30"},
    },
  },
  layer=[
    {
      mark="rect",
      encoding={
        color={
          field="code_size_for_wasabi_over_wastrm",
          type="quantitative",
          scale={
            domainMid=1,
            scheme="redyellowgreen",
            type="log",
          },
          legend={
            title="Wasabi Output Code Size / Wastrumentation Output Code Size",
            orient="top",
            titleLimit=1000,
            gradientLength=540,
            titleAnchor="center",
            titleAlign="center",
          }
        },
      },
    },
    {
      mark={
        type="text",
        fontSize="6",
      },
      encoding={
        text={
          field="relative_overhead_truncated",
          type="nominal",
        },
      },
    },
  ],
  config={
    axis={grid=true, tickBand="extent",}
  },
)

runtime_overhead_comparison_plot |> save("./working-dir/wastrumentation-wasabi-size-overhead-comparison.pdf")
runtime_overhead_comparison_plot

## Plot Engine Warmup Over Time

In [ ]:
# Interprets the time
function interpret_performance_time_unit(performance :: Float64, time_unit ::String)
  if time_unit === "ms"
    performance
  elseif time_unit === "s"
    performance * 1000
  else
    error("Cannot parse $time_unit")
  end
end

# Replace 'error' and 'timeout' with 'missing'
pattern_error   = r"^error$"
pattern_timeout = r"^timeout \d+$"
function replace_parse_error_timeout(df)
  df_parsed = transform(df,
    Cols(:performance, "time-unit") =>
    ByRow((perf, time_unit) -> if occursin(pattern_error, perf)
      [missing, true, false]
    elseif occursin(pattern_timeout, perf)
      [missing, false, true]
    else
      parsed_performance = parse(Float64, perf)
      adjusted_for_unit = interpret_performance_time_unit(parsed_performance, time_unit)
      [adjusted_for_unit, false, false]
    end)
    => [:performance, :error, :timeout]
  )
  select(df_parsed, Not(Cols(["time-unit"])))
end

function parse_time(df)
  df_parsed = transform(df,
    Cols(:performance, "time-unit") =>
    ByRow((perf, time_unit) -> begin
      @assert isa(perf, Float64)
      interpret_performance_time_unit(perf, time_unit)
    end)
    => :performance
  )
  select(df_parsed, Not(Cols(["time-unit"])))
end

df_rgular = DataFrame(load("./working-dir/runtime-analysis-regular.csv")) |> parse_time
df_wasabi = DataFrame(load("./working-dir/runtime-analysis-wasabi.csv")) |> split_setup |> replace_parse_error_timeout
df_wastrm = DataFrame(load("./working-dir/runtime-analysis-wastrumentation.csv")) |> split_setup |> replace_parse_error_timeout

# Squash together inter-runtime iterations => [1, 2, 3, 1, 2, 3] => [1, 2, 3]

function squash_inter_runtime_iterations(df)
  combine(
      groupby(df, Cols(:input_program, :runtime, :runtime_iteration, :platform, :analysis)),
      # :performance::Vec{Float64}, :error::Vec{Bool}, :timeout::Vec{Bool}
      [:performance, :error, :timeout] =>
      # combine rows such that:
      #   if any row has error, the combination is error
      #   if none are error, but any row is timeout, the combination is timeout
      #   none should be error or timeout, the combination is the median
      ((perf, err, to) -> if any(Bool.(err))
          (; performance = missing, error = true, timeout = false)  # Return NamedTuple
      elseif any(Bool.(to))
          (; performance = missing, error = false, timeout = true)  # Return NamedTuple
      else
          (; performance = median(perf), error = false, timeout = false)  # Return NamedTuple
      end)
      => [:performance, :error, :timeout],
  )
end

df_rgular = combine(groupby(df_rgular, Cols(:input_program, :runtime, :runtime_iteration, :setup)), :performance => median => :performance)
df_wasabi = df_wasabi |> squash_inter_runtime_iterations
df_wastrm = df_wastrm |> squash_inter_runtime_iterations

"Parsed all runtime data"

In [ ]:
df_rgular |>
@vlplot(
  :line,
  encoding={
    x={
      field="runtime_iteration",
      type="nominal",
    },
    y={
      aggregate="median",
      field="performance",
      type="quantitative",
      scale={
        type="log"
      },
      title="Execution time (ms)",
    },
    color={
      field="input_program",
      type="nominal",
    },
  },
  config={
    line={
      point=true
    },
    scale={
      useUnaggregatedDomain=true
    },
  },
)


In [ ]:
filter(row -> row.analysis == "forward" && row.platform == "wasabi", df_wasabi) |>
@vlplot(
  :line,
  mark={
      :errorband,
      extent=:ci,
  },
  encoding={
    x={
      field="runtime_iteration",
      type="nominal",
      scale={
        "rangeStep"=12
      },
    },
    y={
      aggregate="median",
      field="performance",
      type="quantitative",
      scale={
        type="log"
      },
      title="Execution time (ms)",
    },
    color={
      field="input_program",
      type="nominal",
    },
  },
  config={
    line={
      point=true
    },
    scale={
      useUnaggregatedDomain=true
    },
  },
)


In [ ]:
filter(row -> row.analysis == "forward" && row.platform == "wastrumentation", df_wastrm) |>
@vlplot(
  :line,
  mark={
      :errorband,
      extent=:ci,
  },
  encoding={
    x={
      field="runtime_iteration",
      type="nominal",
      scale={
        "rangeStep"=12
      },
    },
    y={
      aggregate="median",
      field="performance",
      type="quantitative",
      scale={
        type="log"
      },
      title="Execution time (ms)",
    },
    color={
      field="input_program",
      type="nominal",
    },
  },
  config={
    line={
      point=true
    },
    scale={
      useUnaggregatedDomain=true
    },
  },
)

## Plot Runtime Performances

In [ ]:
df_rgular_median_over_iterations = combine(groupby(df_rgular, Cols(:input_program, :runtime, :setup)), :performance => median => :performance)
df_wasabi_median_over_iterations = combine(groupby(df_wasabi, Cols(:input_program, :runtime, :platform, :analysis)), :performance => median => :performance)
df_wastrm_median_over_iterations = combine(groupby(df_wastrm, Cols(:input_program, :runtime, :platform, :analysis)), :performance => median => :performance)

function squash_iterations(df)
  combine(
    groupby(df, Cols(:input_program, :runtime, :platform, :analysis)),
    # :performance::Vec{Float64}, :error::Vec{Bool}, :timeout::Vec{Bool}
    [:performance, :error, :timeout] =>
    # combine rows such that:
    #   if any row has error, the combination is error
    #   if none are error, but any row is timeout, the combination is timeout
    #   none should be error or timeout, the combination is the median
    ((perf, err, to) -> if any(Bool.(err))
        (; performance = missing, error = true, timeout = false)  # Return NamedTuple
    elseif any(Bool.(to))
        (; performance = missing, error = false, timeout = true)  # Return NamedTuple
    else
        (; performance = median(perf), error = false, timeout = false)  # Return NamedTuple
    end)
    => [:performance, :error, :timeout],
  )
end

df_rgular_median_over_iterations = combine(groupby(df_rgular, Cols(:input_program, :runtime, :setup)), :performance => median => :performance)
df_wasabi_median_over_iterations = df_wasabi |> squash_iterations
df_wastrm_median_over_iterations = df_wastrm |> squash_iterations
"Median over runtimes computed!"

In [ ]:
# E.g. overhead wasabi: 10x
#      overhead wastrm: 50x
#
#      ==> wasabi faster (lower overhead; 10 <= 50)
#
#      ==> wasabi / wastrm = 0.2
#      ==> marked as "1. wasabi is much faster"

performance_ordinal_domain = [
  "1. Wastrmnt >3 times slower",
  "2. Wastrmnt 3-1.05 times slower",
  "3. Wastrmnt comparable",
  "4. Wastrmnt 3-1.05 times faster",
  "5. Wastrmnt >3 times faster",
]

#        1      
# [-∞ ======= 0.3 ======= 0.95 ======= 1.05 ======= 3 ======= 100 ======= ]
function performance_comparison(n::Float64)
    if n >= 0 && n <= 0.3
        return performance_ordinal_domain[1]
    elseif n > 0.3 && n <= 0.95
        return performance_ordinal_domain[2]
    elseif n > 0.95 && n <= 1.05
        return performance_ordinal_domain[3]
    elseif n > 1.05 && n <= 3
        return performance_ordinal_domain[4]
    elseif n > 3 && n < 100
        return performance_ordinal_domain[5]
    elseif n >= 100
        return "0. ❌ wasabi is INCREADIBLY SLOW"
    else
        return "Input is out of range"
    end
  end

function performance_comparison(n::Missing)
    n
  end

In [ ]:
outerjoin(
  select(rename(df_rgular_median_over_iterations, :performance => :rgular_performance), Not([:setup])),
  select(rename(df_wastrm_median_over_iterations, :performance => :wastrm_performance), Not([:platform])),
  on=[:runtime, :input_program],
)

In [ ]:
df_wastrm_instruction_overhead_labeled = transform(
  outerjoin(
    select(rename(df_rgular_median_over_iterations, :performance => :rgular_performance), Not([:setup])),
    select(rename(df_wastrm_median_over_iterations, :performance => :wastrm_performance), Not([:platform])),
    on=[:runtime, :input_program],
  ),
  [:rgular_performance, :wastrm_performance, :error, :timeout] =>
  ByRow((rgular_performance, wastrm_performance, err, timeout) -> 
  begin
    @assert !(rgular_performance === missing)
    @assert !(wastrm_performance === missing && err === missing && timeout === missing)
    if err
        [missing, "E"]
    elseif timeout
      [missing, "T"]
    else
      relative_performance = wastrm_performance / rgular_performance
      [relative_performance, @sprintf("%.4g", relative_performance)]
    end
  end)
  => [:overhead, :text_report]
)


In [ ]:
df_wasabi_instruction_overhead = select(
  rightjoin(rename(select(df_rgular, :input_program, :performance, :runtime_iteration), :performance => :performance_baseline), df_wasabi, on=[:runtime_iteration, :input_program]),
  [:performance, :performance_baseline]
    => ((performance, performance_baseline) -> performance ./ performance_baseline)
    => :overhead,
  # What to keep:
  :input_program, :runtime_iteration, :runtime, :platform, :analysis
)

df_wastrm_instruction_overhead = select(
  rightjoin(rename(select(df_rgular, :input_program, :performance, :runtime_iteration), :performance => :performance_baseline), df_wastrm, on=[:runtime_iteration, :input_program]),
  [:performance, :performance_baseline]
    => ((performance, performance_baseline) -> performance ./ performance_baseline)
    => :overhead,
  # What to keep:
  :input_program, :runtime_iteration, :runtime, :platform, :analysis
)

"Overheads computed!"

In [ ]:
# Aggregate all overhead
df_wasabi_instruction_overhead_single = combine(groupby(df_wasabi_instruction_overhead, Cols(:input_program, :runtime, :platform, :analysis)), :overhead => median => :overhead)
df_wastrm_instruction_overhead_single = combine(groupby(df_wastrm_instruction_overhead, Cols(:input_program, :runtime, :platform, :analysis)), :overhead => median => :overhead)
"Aggregated!"

In [ ]:
wasabi_overhead_plot = df_wasabi_instruction_overhead_single |>
@vlplot(
  "transform"=[
    {
      "calculate"="substring(datum.overhead, 0, 5)",
      "as"="overhead_truncated"
    },
  ],
  encoding={
    y={
      field="analysis",
      type="nominal",
    },
    x={
      field="input_program",
      type="nominal",
      axis={title="Input Program",},
    },
  },
  layer=[
    {
      mark="rect",
      encoding={
        color={
          field="overhead",
          type="quantitative",
          scale={
            type="log",
            scheme="blues",
          },
          legend={
            title="Overhead for Wasabi",
            orient="top",
            titleLimit=1000,
            gradientLength=400,
            titleAnchor="center",
            titleAlign="center",
          }
        },
      },
    },
    {
      mark={
        type="text",
        fontSize="6",
      },
      encoding={
        text={
          field="overhead_truncated",
          type="nominal",
        },
      },
    },
  ],
  config={
    axis={grid=true, tickBand="extent",}
  },
)

# wasabi_overhead_plot |> save("./working-dir/wasabi-overhead.pdf")
wasabi_overhead_plot

In [ ]:
wastrm_overhead_plot_single = df_wastrm_instruction_overhead_labeled |>
@vlplot(
  encoding={
    y={
      field="analysis",
      type="nominal",
      axis={labelAngle="-30"},
    },
    x={
      field="input_program",
      type="nominal",
      axis={labelAngle="-30",title="Input Program",},
    },
  },
  layer=[
    {
      mark="rect",
      encoding={
        color={
          field="overhead",
          type="quantitative",
          scale={
            type="log",
            scheme="blues",
          },
          legend={
            title="Overhead for Wastrumentation",
            orient="top",
            titleLimit=1000,
            gradientLength=540,
            titleAnchor="center",
            titleAlign="center",
          }
        },
      },
    },
    {
      mark={
        type="text",
        fontSize="6",
      },
      encoding={
        text={
          field="text_report",
          type="nominal",
        },
      },
    },
  ],
  config={
    axis={grid=true, tickBand="extent",}
  },
)


wastrm_overhead_plot_single |> save("./working-dir/wastrumentation-runtime-overhead-plot-single.pdf")
wastrm_overhead_plot_single

In [ ]:
wastrm_overhead_plot = df_wastrm_instruction_overhead_labeled |>
@vlplot(
  "spacing"=15,
  "bounds"="flush",
  "vconcat"=[
    {
      "mark"={
        "type"="boxplot",
        "extent"="min-max",
      },
      "height"=100,
      "encoding"={
        "x"={
          field="input_program",
          type="nominal",
          "axis"=false
        },
        "y"={
          field="overhead",
          type="quantitative",
          "scale"={type="log", domainMin=1,},
          "title"="",
        },
      },
    },
    {
      "spacing"=15,
      "bounds"="flush",
      "hconcat"=[
        {
          encoding={
            y={
              field="analysis",
              type="nominal",
              axis={labelAngle="-30"},
            },
            x={
              field="input_program",
              type="nominal",
              axis={title="Input Program",labelAngle="-30"},
            },
          },
          layer=[
            {
              mark="rect",
              encoding={
                color={
                  field="overhead",
                  type="quantitative",
                  scale={
                    type="log",
                    scheme="blues",
                  },
                  legend={
                    title="Overhead for Wastrumentation",
                    orient="top",
                    titleLimit=1000,
                    gradientLength=540,
                    titleAnchor="center",
                    titleAlign="center",
                  }
                },
              },
            },
            {
              mark={
                type="text",
                fontSize="6",
              },
              encoding={
                text={
                  field="text_report",
                  type="nominal",
                },
              },
            },
          ],
        },
        {
          "mark"={
            "type"="boxplot",
            "extent"="min-max",
          },
          "width"=100,
          "encoding"={
            "y"={
              field="analysis",
              type="nominal",
              "axis"=false
            },
            "x"={
              field="overhead",
              type="quantitative",
              "scale"={type="log", domainMin=1,},
              "title"="",
            }
          }
        },
      ]
    }
  ],
  config={
    axis={grid=true, tickBand="extent",}
  },
)

wastrm_overhead_plot |> save("./working-dir/wastrumentation-runtime-overhead-plot.pdf")
wastrm_overhead_plot

In [ ]:
all_performance_overhead = vcat(
  df_wastrm_instruction_overhead_single,
  df_wasabi_instruction_overhead_single,
)

all_performance_overhead_plot = all_performance_overhead |> @vlplot(
  "transform"=[
    {
      "calculate"="substring(datum.overhead, 0, 5)",
      "as"="overhead_truncated"
    },
  ],
  facet={
    row={
      field="platform",
      type="nominal",
    },
  },
  spec={
    layer=[
      {
        mark="rect",encoding={
          y={
            field="analysis",
            type="nominal",
            axis={labelAngle="-30"},
          },
          x={
            field="input_program",
            type="nominal",
            axis={title="Input Program",},
            axis={labelAngle="-30"},
          },
          color={
          field="overhead",
          type="quantitative",
          scale={
            type="log",
            scheme="blues",
          },
          legend={
            title="Runtime Overhead (X)",
            orient="top",
            titleLimit=1000,
            gradientLength=540,
            titleAnchor="center",
            titleAlign="center",
          },
        },
        },
        
      },
      {
        mark={
          type="text",
          fontSize="6",
        },
        encoding={
          y={
            field="analysis",
            type="nominal",
          },
          x={
            field="input_program",
            type="nominal",
            axis={title="Input Program",},
          },
          text={
            field="overhead_truncated",
            type="quantitative",
          },
        },
      },
    ],

  },
  config={
    axis={
      grid=true,
      tickBand="extent",
    },
  },
)


all_performance_overhead_plot |> save("./working-dir/wastrumentation-wasabi-runtime-overhead-single.pdf")
all_performance_overhead_plot

In [ ]:
df_wasabi_wastrm_joined = outerjoin(
    select(rename(df_wasabi_median_over_iterations, Dict(:error => :err_wasabi, :timeout => :to_wasabi, :performance => :perf_wasabi)), Not([:platform])),
    select(rename(df_wastrm_median_over_iterations, Dict(:error => :err_wastrm, :timeout => :to_wastrm, :performance => :perf_wastrm)), Not([:platform])),
    on=[:input_program, :runtime, :analysis],
)

df_wasabi_wastrm_runtime_relative = transform(
    df_wasabi_wastrm_joined,
    [:err_wasabi, :to_wasabi, :perf_wasabi, :err_wastrm, :to_wastrm, :perf_wastrm] =>
    ByRow((err_wasabi, to_wasabi, perf_wasabi, err_wastrm, to_wastrm, perf_wastrm) -> 
    begin
      @assert !(err_wasabi === missing && to_wasabi === missing && err_wastrm === missing && to_wastrm === missing)
      if err_wasabi === missing || to_wasabi === missing
        [missing, "MD-S"]
      elseif err_wastrm === missing || to_wastrm === missing
        [missing, "MD-M"]
      elseif err_wasabi || err_wastrm
        if err_wasabi && err_wastrm
          [missing, "E-M&S"]
        elseif err_wasabi
          [missing, "E-S"]
        elseif err_wastrm
          [missing, "E-M"]
        else
          error("Panic!")
        end
      elseif to_wasabi || to_wastrm
        if to_wasabi && to_wastrm
          [missing, "T-M&S"]
        elseif to_wasabi
          [missing, "T-S"]
        elseif to_wastrm
          [missing, "T-M"]
        else
          error("Panic!")
        end
      else
        @assert ((!err_wasabi) && (!to_wasabi) && (!err_wastrm) && (!to_wastrm))
        relative_performance = perf_wasabi / perf_wastrm
        [relative_performance, @sprintf("%.4g", relative_performance)]
      end
    end)
    => [:wasabi_perf_per_wastrm_perf, :text_report]
)

df_wasabi_wastrm_runtime_relative_plot = df_wasabi_wastrm_runtime_relative |> @vlplot(
  encoding={
    y={
      field="analysis",
      type="nominal",
      axis={labelAngle="-30"},
    },
    x={
      field="input_program",
      type="nominal",
      axis={title="Input Program",labelAngle="-30"},
    },
  },
  layer=[
    {
      mark="rect",
      encoding={
        color={
          field="wasabi_perf_per_wastrm_perf",
          type="quantitative",
          scale={
            type="log",
            domainMid=1,
            scheme="redyellowgreen",
          },
          legend={
            title="Wasabi Execution Time / Wastrumentation Execution Time",
            orient="top",
            titleLimit=1000,
            gradientLength=540,
            titleAnchor="center",
            titleAlign="center",
          },
        },
      },
    },
    {
      mark={
        type="text",
        fontSize="6",
      },
      encoding={
        text={
          field="text_report",
          type="nominal",
        },
      },
    },
  ],
  config={
    axis={grid=true, tickBand="extent",}
  },
)

df_wasabi_wastrm_runtime_relative_plot |> save("./working-dir/df-wasabi-wastrm-runtime-relative.pdf")
df_wasabi_wastrm_runtime_relative_plot

In [ ]:
MD_S = nrow(filter(:text_report => (tr) -> tr .=== "MD-S", df_wasabi_wastrm_runtime_relative))
MD_M = nrow(filter(:text_report => (tr) -> tr .=== "MD-M", df_wasabi_wastrm_runtime_relative))
E_M = nrow(filter(:text_report => (tr) -> tr .=== "E-M", df_wasabi_wastrm_runtime_relative))
E_S = nrow(filter(:text_report => (tr) -> tr .=== "E-S", df_wasabi_wastrm_runtime_relative))
E_MS = nrow(filter(:text_report => (tr) -> tr .=== "E-M&S", df_wasabi_wastrm_runtime_relative))
T_M = nrow(filter(:text_report => (tr) -> tr .=== "T-M", df_wasabi_wastrm_runtime_relative))
T_S = nrow(filter(:text_report => (tr) -> tr .=== "T-S", df_wasabi_wastrm_runtime_relative))
T_MS = nrow(filter(:text_report => (tr) -> tr .=== "T-M&S", df_wasabi_wastrm_runtime_relative))

E_M_filter = nrow(filter((row) -> row.err_wastrm !== missing && row.err_wastrm, df_wasabi_wastrm_runtime_relative))
E_S_filter = nrow(filter((row) -> row.err_wasabi !== missing && row.err_wasabi, df_wasabi_wastrm_runtime_relative))

TO_M_filter = nrow(filter((row) -> row.to_wastrm !== missing && row.to_wastrm, df_wasabi_wastrm_runtime_relative))
TO_S_filter = nrow(filter((row) -> row.to_wasabi !== missing && row.to_wasabi, df_wasabi_wastrm_runtime_relative))

E_M_filter = nrow(filter((row) -> row.err_wastrm !== missing && row.err_wastrm, df_wasabi_wastrm_runtime_relative))
E_S_filter = nrow(filter((row) -> row.err_wasabi !== missing && row.err_wasabi, df_wasabi_wastrm_runtime_relative))

MD_M_filter = nrow(filter((row) -> row.err_wastrm === missing || row.to_wastrm === missing, df_wasabi_wastrm_runtime_relative))
MD_S_filter = nrow(filter((row) -> row.err_wasabi === missing || row.to_wasabi === missing, df_wasabi_wastrm_runtime_relative))

sentence = "
We summarize the labels and occurence per Wastrumentation, Wasabi respectively:
$(MD_M_filter + MD_S_filter) missing data ($MD_M_filter, $MD_S_filter),
$(TO_M_filter + TO_S_filter) timeouts ($TO_M_filter, $TO_S_filter) and
$( E_M_filter +  E_S_filter) uncaught exceptions ($E_M_filter, $E_S_filter).
"

println(sentence)

In [ ]:
df_wasabi_wastrm_runtime_relative

## Let's do the same performance evaluation, but now take the first run!

In [ ]:
# baseline =
#     rename(
#       select(
#         subset(df_rgular, :runtime_iteration => i -> i .== 1),
#         Not([:setup]),
#       ),
#       :performance => :performance_baseline,
#     )

# df_wasabi_timeout_computed = df_wasabi
# df_wastrm_timeout_computed = df_wastrm

# if isa(df_wasabi.performance, Vector{String})
#   df_wasabi_timeout_computed = transform(df_wasabi, :performance => ByRow((x) -> parse(Float64, x == "timeout 10s" ? "10000" : x)) => :performance)
# end
# if isa(df_wastrm.performance, Vector{String})
#   df_wastrm_timeout_computed = transform(df_wastrm, :performance => ByRow((x) -> parse(Float64, x == "timeout 10s" ? "10000" : x)) => :performance)
# end

# @assert isa(df_wasabi_timeout_computed.performance, Vector{Float64}) "df_wasabi should be parsed to Float64"
# @assert isa(df_wastrm_timeout_computed.performance, Vector{Float64}) "df_wasabi should be parsed to Float64"

# using Statistics

# # Aggregate computations per 'run'!
# df_wasabi_aggr = combine(groupby(df_wasabi_timeout_computed, Cols(:runtime_iteration, :setup, :runtime, :input_program, "time-unit")), :performance => median => :performance)
# df_wastrm_aggr = combine(groupby(df_wastrm_timeout_computed, Cols(:runtime_iteration, :setup, :runtime, :input_program, "time-unit")), :performance => median => :performance)

# df_wasabi_sep_analyses = transform(
#   df_wasabi_aggr,
#   :setup => ByRow(setup -> match(r"\[wasabi - (.+)\]", setup).captures[1]) => :analysis,
#   :setup => (_ -> "wasabi") => :setup,
# )
# df_wastrm_sep_analyses = transform(
#   df_wastrm_aggr,
#   :setup => ByRow(setup -> match(r"\[wastrumentation - (.+)\]", setup).captures[1]) => :analysis,
#   :setup => (_ -> "wastrumentation") => :setup,
# )

# df_wasabi_instruction_overhead = select(
#   innerjoin(baseline, df_wasabi_sep_analyses, on=[:input_program, :runtime, :runtime_iteration, "time-unit"]),
#   [:performance, :performance_baseline]
#     => ((performance, performance_baseline) -> performance ./ performance_baseline)
#     => :overhead,
#   # What to keep:
#   :input_program, :setup, :analysis,
# )

# df_wastrm_instruction_overhead = select(
#   innerjoin(baseline, df_wastrm_sep_analyses, on=[:input_program, :runtime, :runtime_iteration, "time-unit"]),
#   [:performance, :performance_baseline]
#     => ((performance, performance_baseline) -> performance ./ performance_baseline)
#     => :overhead,
#   # What to keep:
#   :input_program, :setup, :analysis,
# )

# # Aggregate all overhead
# df_wasabi_instruction_overhead_single = combine(groupby(df_wasabi_instruction_overhead, Cols(:input_program, :setup, :analysis)), :overhead => median => :overhead)
# df_wastrm_instruction_overhead_single = combine(groupby(df_wastrm_instruction_overhead, Cols(:input_program, :setup, :analysis)), :overhead => median => :overhead)

# df_wasabi_wastrm_overhead = transform(
#   innerjoin(
#     rename(select(df_wasabi_instruction_overhead_single, Not(:setup)), :overhead => :overhead_wasabi),
#     rename(select(df_wastrm_instruction_overhead_single, Not(:setup)), :overhead => :overhead_wastrm),
#     on=[:input_program, :analysis]
#   ),
#   [:overhead_wasabi, :overhead_wastrm]
#     => ((overhead_wasabi, overhead_wastrm) -> overhead_wasabi ./ overhead_wastrm)
#     => :time_for_wasabi_per_time_for_wastrm,
# )

# df_wasabi_wastrm_overhead = transform(
#   df_wasabi_wastrm_overhead,
#   :time_for_wasabi_per_time_for_wastrm
#   =>
#   ByRow(performance_comparison)
#   =>
#   :time_for_wasabi_per_time_for_wastrm,
# ) |>
# @vlplot(
#   :rect,
#   encoding={
#     color={
#       field="time_for_wasabi_per_time_for_wastrm",
#       type="ordinal",
#       scale={
#         scheme="blueorange",
#         domain=performance_ordinal_domain
#       },
#     },
#     x={
#       field="analysis",
#       type="nominal",
#     },
#     y={
#       field="input_program",
#       type="nominal",
#     },
#   },
#   config={
#     spacing=100,
#     view={stroke=:transparent},
#     axis={domainWidth=1}
#   },
# )


# Memoization Analysis

In [ ]:
using Pkg
Pkg.add("CSV")

using CSV
using DataFrames
using Dates

In [ ]:
df_memoization_performance = DataFrame(load("./working-dir/memoization_performance_wasmr3.csv"))

In [ ]:
function parse_time_to_ms(time :: String)
  if time === "NA"
    missing
  elseif occursin("ms", time)
    time_element = match(r"(\d+\.?\d*)ms", time).captures[1]
    parse(Float64, time_element)
  elseif occursin("s", time)
    time_element = match(r"(\d+.?\d*)s", time).captures[1]
    parse(Float64, time_element) * 1000
  else
    error("Could not parse $time")
  end
end

# Apply the parsing function to the columns
df_memoization_performance.uninstrumented = parse_time_to_ms.(df_memoization_performance.uninstrumented)
df_memoization_performance.instrumented = parse_time_to_ms.(df_memoization_performance.instrumented)

"Interpretation done"

In [ ]:
df_memoization_performance

In [ ]:
df_memoization_performance_overhead = transform(
    df_memoization_performance,
    [:instrumented, :uninstrumented] => ByRow((instrumented, uninstrumented) ->
        if instrumented !== missing && uninstrumented !== missing
            instrumented / uninstrumented
        else
            missing
        end
    ) => :overhead,
)

df_non_missing_memoization_performance_overhead = filter(row -> row.overhead !== missing, df_memoization_performance_overhead)

df_memoization_performance_overhead_slower = filter(row -> row.overhead > 1, df_non_missing_memoization_performance_overhead)
df_memoization_performance_overhead_faster = filter(row -> row.overhead < 1, df_non_missing_memoization_performance_overhead)
df_memoization_performance_overhead_faster = transform(
    df_memoization_performance_overhead_faster,
    [:instrumented, :uninstrumented] => ByRow((instrumented, uninstrumented) -> uninstrumented / instrumented) => :overhead,
)
"Interpreted all speedups and slowdowns"

In [ ]:
df_memoization_performance_overhead_slower

In [ ]:
df_memoization_performance_overhead_faster

In [ ]:
least_slower = df_memoization_performance_overhead_slower[argmin(df_memoization_performance_overhead_slower.overhead), :]
hghst_slower = df_memoization_performance_overhead_slower[argmax(df_memoization_performance_overhead_slower.overhead), :]
least_slower_prog = least_slower.input_program
least_slower_vlue = least_slower.overhead
hghst_slower_prog = hghst_slower.input_program
hghst_slower_vlue = hghst_slower.overhead

least_faster = df_memoization_performance_overhead_faster[argmin(df_memoization_performance_overhead_faster.overhead), :]
hghst_faster = df_memoization_performance_overhead_faster[argmax(df_memoization_performance_overhead_faster.overhead), :]
least_faster_prog = least_faster.input_program
least_faster_vlue = least_faster.overhead
hghst_faster_prog = hghst_faster.input_program
hghst_faster_vlue = hghst_faster.overhead

sentence = "
Our benchmarks on the memoization analysis show that the analysis can incur a performance penalty but can also speed up the input program execution.
The performance penalty ranges from $(@sprintf("%.2f", least_slower_vlue))x slowdown ($least_slower_prog) to $(@sprintf("%.2f", hghst_slower_vlue))x slowdown ($hghst_slower_prog).
The speed up ranges from $(@sprintf("%.2f", least_faster_vlue))x speedup ($least_faster_prog) to $(@sprintf("%.2f", hghst_faster_vlue))x speedup ($hghst_faster_prog).
"

println(sentence)

In [ ]:
# Split the columns for 'uninstrumented' and 'instrumented' and make them two DF's that we then append to each other
df_memoization_uninstrumented = select(df_memoization_performance, :input_program, :uninstrumented)
df_memoization_instrumented = select(df_memoization_performance, :input_program, :instrumented)

rename!(df_memoization_uninstrumented, :uninstrumented => :runtime)
rename!(df_memoization_instrumented, :instrumented => :runtime)

df_memoization_uninstrumented[!, :type] .= "uninstrumented"
df_memoization_instrumented[!, :type] .= "instrumented"

df_memoization_combined = vcat(df_memoization_uninstrumented, df_memoization_instrumented)
"Merged results into single view"

In [ ]:
df_memoization_combined_type_renamed = transform(df_memoization_combined, :type => ByRow((type) -> begin
    if type .=== "uninstrumented"
        "Regular Execution"
    elseif  type .=== "instrumented"
        "Memoized Execution"
    else
        error("Unknown case in $type")
    end
end) => :type)
"Renamed"

In [ ]:
df_memoization_combined_plot = df_memoization_combined_type_renamed |> @vlplot(
  mark={
      type="bar",
      cornerRadiusEnd=3,  # Adds rounded corners at the top of each bar
  },
  encoding={
    color={
      field="type",
      type="nominal",
      scale={
          scheme="pastel2",  # Color scheme for distinct colors that are accessible
          # range=["instrumented", "uninstrumented"],  # Original values in `type`
          # domain=["Memoized Execution", "Regular Execution"],  # Custom legend names
      },
      legend={
        title="Execution model",
        orient="top",
        titleFontSize=12,
        labelFontSize=10,
      }
    },
    xOffset={
      field="type",
      type="nominal",
    },
    x={
      field="input_program",
      type="nominal",
      axis={
        title="Input Program",
        labelAngle=-30,
        labelFontSize=10,
        titleFontSize=12,
        titlePadding=10,
      },
    },
    y={
      field="runtime",
      type="quantitative",
      axis={
        title="Runtime (ms)",
        titleFontSize=12,
        labelFontSize=10,
        grid=false, # Removes y-axis gridlines for a cleaner look
      },
      scale={
        type="log",
      },
    },
  },
  title="Runtime Comparison for Regular Execution and Memoized Execution",  # Adds a descriptive title
  config={
    axis={
      domainColor="#999",  # Lightens axis lines for a cleaner look
    },
    view={stroke=:transparent},
  }
)

df_memoization_combined_plot |> save("./working-dir/memoization-over-head-plot.pdf")
df_memoization_combined_plot